# Where to place the Next Refugee Shelter?

### _A geographical dashboard of the surrounding on neighbourhoud-level_



Welcome to the code of the product for the course Societal Challenge Product of Masterprogram Engineering And Policy Analysis at the Technical University of Delft. The product is a demo that designed by students. 

The product is a tool that helps with the decision-making process of multiple stakeholders (RVB (Problem-Owner), COA and Municipalities) in the placement of refugee shelters placement the Netherlands. The placement of the shelter is a public-polically sensitive task, as the norms and values do not necessarily allign. The CBS data-base is the basis of the dashboard, where objective properties of the neighbourhouds have been selected. This selection and choice of factors give a overview of the suitability of a neighbourhoud on accessibity of infrastructure, healthcare and commerce. Moreover does it say something about the properties and safety and social cohesion of the neighbourhoud. 

We believe that this tool can help in facilitating a constructive and open conversation on the placement of a refugee shelter with bringing little to no bias into the discussion.

#### The libaries

We have used multiple libraries to programm our product. The following libraries were used:

| Libary | Used for|
|--------|---------|
| pysal | For spatial analysis. Helps analyze patterns in geographic data |
|seaborn | For creating advanced and visually appealing graphs.
|contextily | For adding basemaps to spatial data visualizations. Could for example place your neighborhood data on a map that shows roads, rivers, or city layouts|
|splot | To create visualizations of spatial data. Combins analysis and visualization|
|pandas | For handling and analyzing data. It organizes your data in a clean way which makes it easy to manipulate and analyze|
|folium | For creating interactive maps|
|matplotlib | For creating graphs and plots|
|mapclassify | For classifying data into categories or ranges|

In [ ]:
#libraries
# !pip install pysal
# !pip install seaborn
# !pip install contextily
# !pip install splot
# !pip install pandas
# !pip install folium matplotlib mapclassify
# !pip install pandoc

import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import pandas as pd
from pysal.lib import weights
from pysal.explore import esda
from splot.esda import moran_scatterplot,  plot_local_autocorrelation
import seaborn as sns
import contextily as cx
import matplotlib.colors as mcolors

#### Datasets

(We could not find the right tables in the pakkage, but might be availble in the future. In that case the following code might become usefull):

_"pip install cbsodata
import cbsodata
cbsodata.options.catalog_url ='dataderden.cbs.nl'
tables =cbsodata.get_table_list()
data = pd.DataFrame(tables)
data[data["Title"].str.contains("Wijken")]")_



The following datasets have been used from the CBS. 

| Dataset (Name) | URL|
|--------|---------|
| Kerncijfers_wijken_en_buurten_2023_14122024_150331.csv | https://www.cbs.nl/nl-nl/maatwerk/2024/11/kerncijfers-wijken-en-buurten-2023|
|Leefbaarometer-scores wijken 2002-2022.csv | https://www.leefbaarometer.nl/page/Opendata#scores |
|wijkenbuurten_2022_v3.gpkg | https://www.pdok.nl/ogc-webservices/-/article/cbs-gebiedsindelingen#8787c7edcc0df04094d894518eedea62|



We load the CBS neighborhood data of 2023. The reason for this is that the dataset of 2024 has too many missing values.


In [ ]:
df = pd.read_csv("Kerncijfers_wijken_en_buurten_2023_14122024_150331.csv", delimiter= ";")
df.head(15)

We strip the values of the columns down below so it removes any leading (spaces at the beginning) and trailling (spaces at the end) whitespace characters from the strings.

In [ ]:
df["Wijken en buurten"] = df["Wijken en buurten"].str.strip()
df["Regioaanduiding/Gemeentenaam (naam)"] = df["Regioaanduiding/Gemeentenaam (naam)"].str.strip()
df["Regioaanduiding/Soort regio (omschrijving)"] = df["Regioaanduiding/Soort regio (omschrijving)"].str.strip()
df["Regioaanduiding/Codering (code)"] = df["Regioaanduiding/Codering (code)"].str.strip()

Everything that has 'Wijk' in the column of 'Regioaanduiding/Soort regio (omschrijving) is retrieved. This filters values of the column that are unfindable. 

In [ ]:
df_wijk = df[df["Regioaanduiding/Soort regio (omschrijving)"] == "Wijk"]

In [ ]:
df_wijk.info()

We want the scale to be neighborhood scale so this is the column 'Regioaanduiding/Codering (code)' that gives everything seperate neighborhood a specific code number

In [ ]:
df_wijk["Regioaanduiding/Codering (code)"].unique

We are aggregating only on the neighbourhoud-level.

In [ ]:
df_wijk.head()

From our client the RVB, we received a list of locations we can use for use cases to look at examples for our product. This list is the following:


Regulusweg, Den Haag
Luttenbergerweg, Amsterdam
Neuhuyskade, Den Haag
Brinkstraat, Assen
Strijbeekseweg, Srijbeek
Huiserf, Breda
Rijksweg, Rijen
De Omloop, Zefvenbergschen Hoek
Zwarteweg, Den Haag
Waalresweg, Valkenswaard
Slinge, Rotterdam
Taborstraat, Rotterdam
Rokkeveenseweg, Zoetermeer
PI overmaze, Maastricht
Caissonweg, Almere
PI overmaze, Overloon/ Land van Cuijk

We received the addresses from the RVB. With the zip code we identified the neighborhood names and the ones in the dataset. These neighborhood names are shown down below with the corresponding code number and municipality

In [ ]:
# Define the list of Wijken names in the desired sequence
wijken_list = [
    "Wijk 39 Binckhorst",
    "Amstel III/Bullewijk",
    "Wijk 04 Benoordenhout",
    "Assen-Oost",
    "Galder",
    "Breda noord-west",
    "Rijen",
    "Wijk 01 Zevenbergschen Hoek",
    "Wijk 11 Duinoord",
    "Wijk 00 Valkenswaard",
    "Charlois",
    "Kralingen-Crooswijk",
    "Industriegebied",
    "Wijk 04 Buitenwijk Oost",
    "Centrum Almere Buiten",
    "Cuijk"
]

# Filter the DataFrame to include only rows with the Wijken in the list
filtered_df = df_wijk[df_wijk["Wijken en buurten"].isin(wijken_list)]

# Ensure the order of rows matches the order of the Wijken in the list
filtered_df = filtered_df.set_index("Wijken en buurten").loc[wijken_list]

# Reset index to make it cleaner
result = filtered_df.reset_index()

# Select relevant columns
result = result[["Wijken en buurten", "Regioaanduiding/Codering (code)", "Regioaanduiding/Gemeentenaam (naam)"]]


print(result)


Now we load the dataset of the 'leefbarometer'. Note that this set is from 2022 and is a limitation of our analysis. The reason for this is that the 'leefbarometer' is only measured once every two years. 

In [ ]:
df_baro = pd.read_csv("Leefbaarometer-scores wijken 2002-2022.csv")

In [ ]:
df_baro_2022 = df_baro[df_baro["jaar"] == 2022]
df_baro.head(15)

We now merge the two datasets based on their neighborhood.


In [ ]:
df_combined = df_wijk.merge(df_baro_2022, left_on="Regioaanduiding/Codering (code)", right_on="wk_code", how = "left")

Now we add a SHP file to give the values and neighborhood code also a spatial value

In [ ]:
#CSV and add SHP file (geographical data)
gdf= gpd.read_file("wijkenbuurten_2022_v3.gpkg")
gdf.head(5)

We filter the dataset so only values that include 'WK' in the string of 'wijkcode' are included in the dataset

In [ ]:
gdf = gdf[gdf["wijkcode"].str.contains("WK")]   

In [ ]:
gdf["wijkcode"].unique

We add the geodata to eventually have the final dataset. This dataset still can be filtered but has the aspects we want in our dataset.

In [ ]:
Df_final = pd.merge(gdf, df_combined, how='left', left_on="wijkcode",
                right_on="Regioaanduiding/Codering (code)")


In [ ]:
Df_final.isna().sum()

In [ ]:
#Df_final.head(5)

Here we are able to download this dataset as 'output.xlsx'

In [ ]:
print(Df_final.columns)
Df_final.head(1).to_excel('output.xlsx', header=True, index=False, na_rep='null')

Now what is important is that not every variable from the huge dataset is needed. In our product we choose 10 variables. The variables are

'aantal_huishoudens',

'aantal_bedrijfsvestigingen', 

'ziekenhuis_excl_buitenpolikliniek_gem_afst_in_km',

'oprit_hoofdverkeersweg_gemiddelde_afstand_in_km',

'grote_supermarkt_gemiddelde_afstand_in_km',

'basisonderwijs_gemiddelde_afstand_in_km',

'voortgezet_onderwijs_gem_afstand_in_km',

'treinstation_gemiddelde_afstand_in_km',

'soc',

'onv'

The explanation of the first 8 variables can be found at the CBS website.
The explanation of 'soc' and 'onv' can be found here: https://leefbaarometer.nl/page/dimensies 

In [ ]:
Df_Final_filtered = Df_final.loc[:, Df_final.columns.isin([
    'wijkcode',
    'indelingswijziging_wijken_en_buurten',
    'geometry',
    'aantal_huishoudens',
    'aantal_bedrijfsvestigingen',
    'ziekenhuis_excl_buitenpolikliniek_gem_afst_in_km',
    'oprit_hoofdverkeersweg_gemiddelde_afstand_in_km',
    'grote_supermarkt_gemiddelde_afstand_in_km',
    'basisonderwijs_gemiddelde_afstand_in_km',
    'voortgezet_onderwijs_gem_afstand_in_km',
    'treinstation_gemiddelde_afstand_in_km',
    'soc',
    'onv'  
])]

Here this code extracts the names of all columns in the dataframe starting from the 66th column except for the geometry columns. This gives all the variables we want as column names.

In [ ]:
Column_names_1 = [column for column in Df_Final_filtered.iloc[:,2:]]
Column_names = [column for column in Column_names_1 if column != "geometry"]
print(Column_names)

In [ ]:
Df_Final_filtered = Df_Final_filtered.copy()

In [ ]:
Df_Final_filtered.describe()

We note that some values are (well below 0). We clean them and this results in values below zero are also excluded

In [ ]:
columns_to_clean = [
    "aantal_huishoudens",
    "aantal_bedrijfsvestigingen",
    "grote_supermarkt_gemiddelde_afstand_in_km",
    "basisonderwijs_gemiddelde_afstand_in_km",
    "ziekenhuis_excl_buitenpolikliniek_gem_afst_in_km",
    "oprit_hoofdverkeersweg_gemiddelde_afstand_in_km",
    "voortgezet_onderwijs_gem_afstand_in_km",
    "treinstation_gemiddelde_afstand_in_km",
    "onv",
    "soc"
]

In [ ]:
# Replace invalid values with NaN
for col in columns_to_clean:
    Df_Final_filtered[col] = Df_Final_filtered[col].where(Df_Final_filtered[col] >-1000, other=-1000)
    Df_Final_filtered = Df_Final_filtered[Df_Final_filtered[col] != -1000].to_crs(epsg=3857)
Df_Final_filtered.describe()

In [ ]:
import folium
List_of_RefugeeShelters = ["WK051839","WK051804","WK010603","WK172304","WK075806","WK078401",
                           "WK085800","WK059915","WK059908","WK063708","WK093504","WK003441","WK198208"]
Df_Final_filtered["RefugeeShelter"] = np.where(Df_Final_filtered["wijkcode"].isin(List_of_RefugeeShelters),1,0)


In [ ]:
# Rename columns
Df_Final_filtered = Df_Final_filtered.rename(columns={
    'wijkcode':"Neigbourhoud Code (CBS)",
    "aantal_huishoudens":'Number of households',
    "aantal_bedrijfsvestigingen": 'Number of business establishments',
    "grote_supermarkt_gemiddelde_afstand_in_km": 'Accessibility for Large Supermarket (km)',
    "basisonderwijs_gemiddelde_afstand_in_km": 'Accessibility for Primary Education (km)',
    "ziekenhuis_excl_buitenpolikliniek_gem_afst_in_km": 'Accessibility to Hospitals (Excl. Outpatient Clinic) (km)',
    "oprit_hoofdverkeersweg_gemiddelde_afstand_in_km": 'Accessibility to main road (km)',
    "voortgezet_onderwijs_gem_afstand_in_km": 'Accessibility for Secondary Education (km)',
    "treinstation_gemiddelde_afstand_in_km": 'Train Station Accessibility (km)',
    "onv": 'Leefbaroscore Disturbance and lack of safety',
    "soc": "Leefbaroscore Social Cohesion"})

In [ ]:

variable_settings = {
    'Accessibility for Large Supermarket (km)':{ 
        'thresholds': { '≤ 2.0 km': 2.0,'2.0 - 3.25 km': 3.25},
        'colors': {'≤ 2.0 km': 'green','2.0 - 3.25 km': 'yellow','> 3.25 km': 'lightcoral' },
        'title': 'Accessibility for Large Supermarket (km)'},
    
    'Accessibility for Primary Education (km)':{ 
        'thresholds': {'≤ 1.25 km': 1.25,'1.25 - 2.5 km': 2.5},
        'colors': {'≤ 1.25 km': 'green','1.25 - 2.5 km': 'yellow','> 2.5 km': 'lightcoral'},
        'title': 'Accessibility for Primary Education (km)'},
    
      'Accessibility to main road (km)':{ 
        'thresholds': {'≤ 2 km': 2,'2 - 5 km': 5},
        'colors': {'≤ 2 km': 'green','2 - 5 km': 'yellow','> 5 km': 'lightcoral'},
        'title': 'Accessibility to main road (km)'},
    
    'Accessibility for Secondary Education (km)':{ 
         'thresholds': {'≤ 7.5 km': 7.5,'7.5 - 15 km': 15.0},
        'colors': { '≤ 7.5 km': 'green','7.5 - 15 km': 'yellow','> 15 km': 'lightcoral'},
        'title': 'Accessibility for Secondary Education (km)'},
    
    'Train Station Accessibility (km)' :{ 
        'thresholds': {'≤ 7.5 km':7.5, '7.5 - 15 km': 15},
        'colors': { '≤ 7.5 km': 'green','7.5 - 15 km': 'yellow','> 15 km': 'lightcoral'},
        'title': 'Train Station Accessibility (km)'},
    
    'Accessibility to Hospitals (Excl. Outpatient Clinic) (km)' :{ 
        'thresholds': {'≤ 5 km':5, '5 - 15 km': 15},
        'colors': { '≤ 5 km': 'green','5 - 15 km': 'yellow','> 15 km': 'lightcoral'},
        'title': 'Accessibility to Hospitals (Excl. Outpatient Clinic) (km)'}
    
}
 

# Function to classify values into categories based on thresholds
def classify_value(value, thresholds):
    if np.isnan(value):  # Handle missing values
        return 'Missing'
    threshold_values = list(thresholds.values())
    threshold_keys = list(thresholds.keys())
    if value <= threshold_values[0]:
        return threshold_keys[0]  # First category, e.g., "≤ 2.0 km"
    elif value <= threshold_values[1]:
        return threshold_keys[1]  # Second category, e.g., "2.0 - 3.25 km"
    else:
        return f'>{threshold_keys[1].split("-")[-1]}'.strip()  # Third category, e.g., "> 3.25 km"

# Iterate over each variable and generate individual maps
for column, settings in variable_settings.items():
    thresholds = settings['thresholds']
    colors = settings['colors']
    map_title = settings['title']
    
    # Create a new column for the category
    Df_Final_filtered[f'{column}_category'] = Df_Final_filtered[column].apply(classify_value, args=(thresholds,))
  
    
    # Map categories to colors
    Df_Final_filtered['color'] = Df_Final_filtered[f'{column}_category'].map(colors)
    
    
    # Plot the map
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    Df_Final_filtered.plot(
        ax=ax,
        color=Df_Final_filtered['color'],
        edgecolor='black',
        linewidth=0.2
    )
    
    # Add title and legend
    ax.set_title(map_title, fontsize=14)
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color='w', label=cat, markerfacecolor=color, markersize=10)
        for cat, color in colors.items()
    ]
    ax.legend(handles=legend_elements, loc='lower left', title="Categories")
    # Show the map
    plt.tight_layout()
    plt.plot()

In [ ]:
# Define the column to plot
column_to_plot = 'Number of households'

# Define the colors for the quantiles
colors = ['green', 'yellow', 'lightcoral']  # Low = green, Mid = yellow, High = red
cmap = mcolors.ListedColormap(colors)

# Create the plot
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
Df_Final_filtered.plot(
    column=column_to_plot,
    legend=True,
    scheme="quantiles",  # Quantiles scheme
    cmap = mcolors.ListedColormap(colors),  # Custom color map
    k=3,  # Number of quantiles
    ax=ax,
    edgecolor="black",
    linewidth=0.2
)

# Add title and adjust layout
ax.set_title('Number of households', fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:

# Define the column to plot
column_to_plot = 'Number of business establishments'

# Define the colors for the quantiles
colors = ['lightcoral', 'yellow', 'green']  # Low = red, Mid = yellow, High = green
cmap = mcolors.ListedColormap(colors)

# Create the plot
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
Df_Final_filtered.plot(
    column=column_to_plot,
    legend=True,
    scheme="quantiles",  # Quantiles scheme
    cmap=mcolors.ListedColormap(colors),  # Custom color map
    k=3,  # Number of quantiles
    ax=ax,
    edgecolor="black",
    linewidth=0.2
)

# Add title and adjust layout
ax.set_title('Number of business establishments', fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
# Define the column to plot
column_to_plot = 'Leefbaroscore Disturbance and lack of safety'

# Define colors for quantiles
colors = ['lightcoral', 'yellow', 'green']  # Low = red, Mid = yellow, High = green
cmap = mcolors.ListedColormap(colors)

# Calculate quantiles
quantiles = Df_Final_filtered[column_to_plot].quantile([0, 0.33, 0.66, 1]).values

# Create a new column to classify values based on quantiles
def classify_quantiles(value):
    if value <= quantiles[1]:  # First quantile range
        return 'Low'
    elif value <= quantiles[2]:  # Second quantile range
        return 'Medium'
    else:  # Third quantile range
        return 'High'

Df_Final_filtered['Leefbaroscore Disturbance and lack of safety_category'] = Df_Final_filtered[column_to_plot].apply(classify_quantiles)

# Map the categories to colors
Df_Final_filtered['color'] = Df_Final_filtered['Leefbaroscore Disturbance and lack of safety_category'].map({
    'Low': 'lightcoral',
    'Medium': 'yellow',
    'High': 'green'
})

# Plot the map
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
Df_Final_filtered.plot(
    color=Df_Final_filtered['color'],
    edgecolor='black',
    ax=ax,
    linewidth=0.2,
    missing_kwds={"color": "lightgrey", "label": "No Data"}
)

# Add title and legend
ax.set_title( 'Leefbaroscore Disturbance and lack of safety', fontsize=14)
legend_elements = [
    plt.Line2D([0], [0], marker='o', color='w', label=f'≤ {quantiles[1]:.2f}', markerfacecolor='lightcoral', markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label=f'{quantiles[1]:.2f} - {quantiles[2]:.2f}', markerfacecolor='yellow', markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label=f'> {quantiles[2]:.2f}', markerfacecolor='green', markersize=10)
]
ax.legend(handles=legend_elements, loc='lower left')

# Adjust layout and show
plt.tight_layout()
plt.show()


In [ ]:
# Define the column to plot
column_to_plot = 'Leefbaroscore Social Cohesion'

# Define colors for quantiles
colors = ['lightcoral', 'yellow', 'green']  # Low = red, Mid = yellow, High = green
cmap = mcolors.ListedColormap(colors)

# Calculate quantiles
quantiles = Df_Final_filtered[column_to_plot].quantile([0, 0.33, 0.66, 1]).values

# Create a new column to classify values based on quantiles
def classify_quantiles(value):
    if value <= quantiles[1]:  # First quantile range
        return 'Low'
    elif value <= quantiles[2]:  # Second quantile range
        return 'Medium'
    else:  # Third quantile range
        return 'High'

Df_Final_filtered['Leefbaroscore Social Cohesion_category'] = Df_Final_filtered[column_to_plot].apply(classify_quantiles)


# Plot the map
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
Df_Final_filtered.plot(
    color=Df_Final_filtered['color'],
    edgecolor='black',
    ax=ax,
    cmap=mcolors.ListedColormap(colors),
    linewidth=0.2,
    missing_kwds={"color": "blue", "label": "No Data"}
)

# Add title and legend
ax.set_title( 'Leefbaroscore Social Cohesion', fontsize=14)
legend_elements = [
    plt.Line2D([0], [0], marker='o', color='w', label=f'≤ {quantiles[1]:.2f}', markerfacecolor='lightcoral', markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label=f'{quantiles[1]:.2f} - {quantiles[2]:.2f}', markerfacecolor='yellow', markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label=f'> {quantiles[2]:.2f}', markerfacecolor='green', markersize=10)
]
ax.legend(handles=legend_elements, loc='lower left')

# Adjust layout and show
plt.tight_layout()
plt.show(block=True)


After showing all the variables on the map seperately. We not use the explore function to give every factor we included into a list when you click on a specific area.

In [ ]:
Df_Final_filtered = Df_Final_filtered.loc[:, Df_Final_filtered.columns.isin([
                                    "Neigbourhoud Code (CBS)",
                                    'Number of households',
                                    'Number of business establishments',
                                   'Accessibility for Large Supermarket (km)',
                                   'Accessibility for Primary Education (km)',
                                   'Accessibility to Hospitals (Excl. Outpatient Clinic) (km)',
                                   'Accessibility to main road (km)',
                                   'Accessibility for Secondary Education (km)',
                                   'Train Station Accessibility (km)',
                                   'Leefbaroscore Disturbance and lack of safety',
                                   "Leefbaroscore Social Cohesion",
                                    'RefugeeShelter',
                                    'geometry'])]
Df_Final_filtered.head(10)
Df_Final_filtered.to_crs(epsg=3857)

In [ ]:
colors = ['lightcoral', 'yellow', 'green']  # Low = red, Mid = yellow, High = green
cmap = mcolors.ListedColormap(colors)


#Df_Final_filtered.explore(column="Number of business establishments",cmap= "RdYlGn_r",scheme="quantiles")
#Df_Final_filtered.explore(column="Number of households',cmap= "RdYlGn_r",scheme="quantiles")
#Df_Final_filtered.explore(column="Accessibility for Large Supermarket (km)",cmap= "RdYlGn_r",scheme="quantiles")
#Df_Final_filtered.explore(column="Accessibility for Primary Education (km)",cmap= "RdYlGn_r",scheme="quantiles")
#Df_Final_filtered.explore(column="Accessibility to Hospitals (Excl. Outpatient Clinic) (km)",cmap= "RdYlGn_r",scheme="quantiles")
#Df_Final_filtered.explore(column="Accessibility to main road (km)",cmap= "RdYlGn_r",scheme="quantiles")
#Df_Final_filtered.explore(column="Accessibility for Secondary Education (km)",cmap= "RdYlGn_r",scheme="quantiles")
#Df_Final_filtered.explore(column="Train Station Accessibility (km)",cmap= "RdYlGn_r",scheme="quantiles")
#Df_Final_filtered.explore(column="Leefbaroscore Disturbance and lack of safety'",cmap= "RdYlGn_r",scheme="quantiles")
#Df_Final_filtered.explore(column="Leefbaroscore Social Cohesion",cmap= "RdYlGn_r",scheme="quantiles")

titles     co
'Leefbaroscore Social Cohesion' 